In [2]:
import os
import random
import numpy as np
import pandas as pd
from glob import glob
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input

# Config
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 15,
    'LR': 3e-4,
    'BATCH_SIZE': 24,
    'SEED': 2025,
    'FOLDS': 5
}

# Seed 고정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(CFG['SEED'])

# 데이터 경로
TRAIN_DIR = "D:/데이콘 250519 대회/open/train"
TEST_DIR = "D:/데이콘 250519 대회/open/test"
SAMPLE_SUB = "D:/데이콘 250519 대회/open/sample_submission.csv"

# 라벨 매핑
label_list = sorted(os.listdir(TRAIN_DIR))
label2id = {v: i for i, v in enumerate(label_list)}
id2label = {i: v for v, i in label2id.items()}

# 이미지 및 라벨 로딩
image_paths = glob(os.path.join(TRAIN_DIR, '*', '*.jpg'))
labels = [label2id[os.path.basename(os.path.dirname(p))] for p in image_paths]

# 이미지 전처리 함수
def load_and_preprocess(img_path):
    img = load_img(img_path, target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

# TF Dataset 생성
def create_dataset(image_paths, labels=None, is_train=True):
    def gen():
        for i, path in enumerate(image_paths):
            img = load_and_preprocess(path)
            if labels is not None:
                yield img, labels[i]
            else:
                yield img

    if labels is not None:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=(tf.float32, tf.int32),
            output_shapes=((CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), ())
        )
    else:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=tf.float32,
            output_shapes=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3)
        )

    if is_train:
        ds = ds.shuffle(1024)
    ds = ds.batch(CFG['BATCH_SIZE']).prefetch(tf.data.AUTOTUNE)
    return ds


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import StratifiedKFold

# EfficientNet 기반 모델 생성
def build_model(num_classes):
    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        input_shape=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3),
        weights='imagenet',
        pooling='avg'
    )
    base.trainable = True  # 모든 레이어 학습 (GPU 성능 활용)

    x = layers.Dense(256, activation='relu')(base.output)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=base.input, outputs=output)
    return model

# Stratified K-Fold 설정
skf = StratifiedKFold(n_splits=CFG['FOLDS'], shuffle=True, random_state=CFG['SEED'])

best_val_acc = 0
best_model_path = "best_model_weights.h5"

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"\n### Fold {fold+1}")

    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]

    train_ds = create_dataset(train_paths, train_labels, is_train=True)
    val_ds = create_dataset(val_paths, val_labels, is_train=False)

    model = build_model(num_classes=len(label2id))
    model.compile(
        optimizer=optimizers.Adam(CFG['LR']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(
            f"fold{fold+1}_best_weights.h5", 
            save_best_only=True,
            save_weights_only=True,  # JSON 오류 방지
            monitor="val_accuracy", 
            mode="max"
        ),
        tf.keras.callbacks.EarlyStopping(
            monitor="val_accuracy", patience=3, restore_best_weights=True
        )
    ]

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=CFG['EPOCHS'],
        callbacks=callbacks,
        verbose=1
    )

    val_acc = max(history.history['val_accuracy'])
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        model.save_weights(best_model_path)  # 가중치만 저장

print(f"\n✅ 최고 검증 정확도: {best_val_acc:.4f}")




### Fold 1
Epoch 1/15
1105/1105 [==============================] - 108s 90ms/step - loss: 5.4202 - accuracy: 0.0837 - val_loss: 6.8076 - val_accuracy: 0.0115
Epoch 2/15
1105/1105 [==============================] - 103s 91ms/step - loss: 2.9113 - accuracy: 0.4162 - val_loss: 6.5617 - val_accuracy: 0.1017
Epoch 3/15
1105/1105 [==============================] - 103s 91ms/step - loss: 1.0476 - accuracy: 0.7344 - val_loss: 5.6587 - val_accuracy: 0.1886
Epoch 4/15
1105/1105 [==============================] - 100s 88ms/step - loss: 0.5447 - accuracy: 0.8506 - val_loss: 5.1726 - val_accuracy: 0.2488
Epoch 5/15
1105/1105 [==============================] - 99s 88ms/step - loss: 0.3588 - accuracy: 0.8997 - val_loss: 4.9013 - val_accuracy: 0.2835
Epoch 6/15
1105/1105 [==============================] - 101s 89ms/step - loss: 0.2624 - accuracy: 0.9252 - val_loss: 4.3940 - val_accuracy: 0.3378
Epoch 7/15
1105/1105 [==============================] - 100s 88ms/step - loss: 0.2236 - accuracy: 0.9327 - 

In [ ]:
# Confusion Matrix 시각화 함수
def plot_confusion_matrix(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

# 검증 데이터 예측 후 시각화
val_preds = model.predict(val_ds)
val_pred_labels = np.argmax(val_preds, axis=1)

plot_confusion_matrix(val_labels, val_pred_labels, list(label2id.keys()))


In [ ]:
import cv2

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)
    return heatmap.numpy()

# Grad-CAM 시각화 함수
def display_gradcam(img_path, model, last_conv_layer_name='top_conv'):
    img = load_img(img_path, target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

    # 원본 이미지 준비
    img_original = cv2.imread(img_path)
    img_original = cv2.resize(img_original, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)

    # heatmap 이미지 생성 및 중첩
    heatmap = cv2.resize(heatmap, (img_original.shape[1], img_original.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img_original, 0.6, heatmap_color, 0.4, 0)

    # 시각화
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 1)
    plt.title("Original")
    plt.imshow(img_original)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title("Grad-CAM")
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# 예시: 확인하고 싶은 이미지 경로 지정
display_gradcam(image_paths[0], model)


In [ ]:
import pandas as pd
import numpy as np

# 1. 제출 템플릿 불러오기
submission = pd.read_csv('./sample_submission.csv', encoding='utf-8-sig')

# 2. 클래스 컬럼만 추출 (예: ['class_A', 'class_B', ..., 'class_N'])
class_columns = submission.columns[1:]

# 3. 모델 예측 (확률 값으로 반환됨, 예: (N, C))
pred_prob = model.predict(test_ds)

# 4. 예측 확률 → DataFrame 변환 + 컬럼 정렬
pred_df = pd.DataFrame(pred_prob, columns=class_columns)

# 5. 예측 결과를 제출 템플릿에 반영
submission[class_columns] = pred_df[class_columns].values

# 6. 저장
submission.to_csv('baseline_submission.csv', index=False, encoding='utf-8-sig')

print("✅ 예측 결과가 baseline_submission.csv로 저장되었습니다.")
